# Quantum-Chemical Analysis of Asphalt Oxidation Energies

## Imports & Data Loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Load quantum results (VQE optimizations)
quantum = pd.read_csv('../results/optim_grid_tmp1.csv')

# Load classical reference data
classical = pd.read_csv('../data/classical_energies.csv')

## Energy Comparison

In [2]:
# Get best quantum result (lowest energy)
best_q = quantum.nsmallest(1, 'Final energy (Ha)').iloc[0]
q_dE = best_q['Final energy (Ha)']
q_dE_kj = q_dE * 2625.5

# Classical references
ccsd = classical[classical['Method']=='CCSD(T)'].iloc[0]
dft = classical[classical['Method']=='ωB97X-D'].iloc[0]

# Comparison dataframe
results = pd.DataFrame({
    'Method': ['Quantum', 'CCSD(T)', 'DFT'],
    'ΔE_ox (Ha)': [q_dE, ccsd['dE_ox[Ha]'], dft['dE_ox[Ha]']],
    'ΔE_ox (kJ/mol)': [q_dE_kj, ccsd['dE_ox[Ha]']*2625.5, dft['dE_ox[Ha]']*2625.5],
    'Wall-time (s)': [best_q['Wall-time (s)'], ccsd['CPU_time[s]'], dft['CPU_time[s]']]
})

## Error Assessment

In [3]:
# Calculate errors vs CCSD(T)
ccsd_ref = results[results['Method']=='CCSD(T)']['ΔE_ox (Ha)'].values[0]
results['Error (mHa)'] = (results['ΔE_ox (Ha)'] - ccsd_ref) * 1000
results['Error Flag'] = np.abs(results['Error (mHa)']) > 15

print("Energy Comparison:")
display(results)

Energy Comparison:


,Method,ΔE_ox (Ha),ΔE_ox (kJ/mol),Wall-time (s),Error (mHa),Error Flag
0,Quantum,-1871.128727,-4.912648e+06,970.02,-1.796153e+06,True
1,CCSD(T),-74.975584,-1.968484e+05,8525.00,0.000000e+00,False
2,DFT,-75.134398,-1.972654e+05,200.00,-1.588132e+02,True


## Industrial Impact

In [4]:
# Lifetime extension calculation
dE_diff = (results.loc[0,'ΔE_ox (kJ/mol)'] - results.loc[1,'ΔE_ox (kJ/mol)'])
rate_ratio = np.exp(-dE_diff / (8.314 * 298))
life_gain = (1/rate_ratio - 1) * 15

# Cost savings (mid-range FHWA estimate)
savings = 40 * (life_gain / 15)

print(f"Estimated lifetime extension: {life_gain:.1f} years")
print(f"Potential annual savings: ${savings:.2f} billion")

Estimated lifetime extension: -15.0 years
Potential annual savings: $-40.00 billion


/var/folders/dq/xfbc1m4x5w7g74nwsyj7yb4m0000gn/T/ipykernel_86087/1251418367.py:3: RuntimeWarning: overflow encountered in exp
  rate_ratio = np.exp(-dE_diff / (8.314 * 298))


## Visualization

In [5]:
plt.style.use('seaborn-v0_8')
fig, ax = plt.subplots(figsize=(8,5))

bars = ax.bar(results['Method'], results['ΔE_ox (kJ/mol)'], 
              color=['#4C72B0', '#DD8452', '#55A868'])

# Annotate with compute times
for bar, time in zip(bars, results['Wall-time (s)']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height()/2,
            f'{time:.0f} s', ha='center', va='center',
            color='white', fontweight='bold')

ax.set_ylabel('ΔEₒₓ (kJ/mol)', fontsize=12)
ax.set_title('Oxidation Energy Barrier Comparison', pad=20)
plt.tight_layout()
plt.savefig('../results/figure1_deltaE.png', dpi=300, bbox_inches='tight')